In [29]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [30]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(QQ)

In [31]:
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)

In [32]:
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [33]:
line = Line([y, z])

In [34]:
general_cubic = Cubic(cubic_new, line, sing_cubics)

In [35]:
SE4 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({f:c*(3*c-e)/(c+e)})

In [36]:
adm_perm = SE4.find_admissible_permutations()
len(adm_perm)

72

In [37]:
%time adm_SE4 = SE4.find_admissible_projectivities()

CPU times: user 428 ms, sys: 96.4 ms, total: 524 ms
Wall time: 910 ms


In [38]:
len(adm_SE4)

36

In [39]:
%time simm_SE4 = SE4.find_simmetries(adm_SE4)

CPU times: user 195 ms, sys: 58.2 ms, total: 254 ms
Wall time: 292 ms


In [40]:
len(simm_SE4)

12

In [ ]:
G_SE4 = Group(simm_SE4)

In [ ]:
G_SE4.is_group()

In [19]:
G_SE4.is_abelian()

NameError: name 'G_SE4' is not defined

In [20]:
G_SE4.get_order_of_elements()

NameError: name 'G_SE4' is not defined

In [17]:
SE4.find_conditions_for_subfamilies(adm_SE4, simm_SE4)

[]

In [18]:
perms = []
for el in G_SE4:
    perms.append(Permutation(apply_proj_to_eck(el, SE4.eckardt_points)).to_cycles())

In [19]:
gp = PermutationGroup(perms)
gp.order()

6

In [20]:
gp.structure_description()

'S3'

In [21]:
keys = list(SE4.cl_lines.keys())

In [22]:
perms_lines = []
for simm in simm_SE4:
    perms_lines.append(Permutation(apply_proj_to_lines(simm, list(SE4.cl_lines.values()))).to_cycles())

In [23]:
Gr = PermutationGroup(perms_lines)

In [24]:
Gr.order()

12

In [25]:
Gr.structure_description()

'D6'

In [26]:
Gr.minimal_generating_set()

[(2,19)(3,22)(4,7)(5,25)(6,26)(8,13)(9,14)(11,16)(12,17)(18,27)(20,24)(21,23),
 (1,18,14,15,9,27)(2,19,10)(3,7,4,22,13,8)(5,12,17,25,21,23)(6,11,16,26,20,24)]

In [27]:
[el.is_isomorphic(SymmetricGroup(3)) for el in Gr.normal_subgroups()]

[False, True, True, False, False, False, False]

In [28]:
nor_sub = Gr.normal_subgroups()[2]
nor_sub.minimal_generating_set()

[(1,15)(2,19)(4,13)(7,8)(9,27)(11,24)(12,23)(14,18)(16,20)(17,21),
 (1,9,14)(2,19,10)(3,13,4)(5,21,17)(6,20,16)(7,8,22)(11,24,26)(12,23,25)(15,18,27)]

In [29]:
Gr.quotient(nor_sub).list()

[(), (1,2)]